In [115]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
from pymc.math import exp, ge

In [73]:
data = pd.read_csv("bladderc.csv")

In [74]:
data.head(20)

,time,observed,group
0,0,0,0
1,1,0,0
2,4,0,0
3,7,0,0
4,10,0,0
5,6,1,0
6,14,0,0
7,18,0,0
8,5,1,0
9,12,1,0


So if we oberved the tumor, we want to right-censor the data. If not, we don't.

In [116]:
data["censored"] = 0
data.loc[data["observed"] == 0, "censored"] = data["time"]

y = data["time"].to_numpy(copy=True)
x = data["group"].to_numpy(copy=True)
observed = data["observed"].to_numpy(copy=True).astype(bool)
censored = data["censored"].to_numpy(copy=True)

np.sum(observed), y.shape, x.shape, observed.shape, censored.shape

(47, (86,), (86,), (86,), (86,))

In [117]:
x_uncensored = x[observed]
x_censored = x[~observed]

In [119]:
y_uncensored = y[observed]

In [120]:
censored = censored[~observed]

In [122]:
# right-censored model

with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, sigma=10)
    beta1 = pm.Normal("beta1", 0, sigma=10)
    
    λ_uncensored = exp(beta0 + beta1 * x_uncensored)
    λ_censored = exp(beta0 + beta1 * x_censored)
    
    likelihood_uncensored = pm.Exponential("likelihood_uncensored", λ_uncensored, observed=y_uncensored, shape=y_uncensored.shape[0])
    likelihood_censored = pm.Bound("censored", pm.Exponential.dist(λ_censored), lower=censored, shape=censored.shape[0])

    mu_placebo = pm.Deterministic("mu_placebo", exp(-beta0))
    mu_chemo = pm.Deterministic("mu_chemo", exp(-beta0 - beta1))

    mu_diff = pm.Deterministic("mu_diff", mu_chemo - mu_placebo)

    H_prob = pm.Deterministic("H_prob", switch(ge(mu_diff, 0), 1, 0))

    trace = pm.sample(
        10000, tune=2000, cores=4, init="auto"
    )

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/sampling.py:635: UserWarning: Unable to validate shapes: Cannot sample from a bounded variable
Apply node that caused the error: bound_rv{0, (0, 0, 0), floatX, False}(RandomStateSharedVariable(<RandomState(MT19937) at 0x129E04C40>), TensorConstant{(1,) of 39}, TensorConstant{11}, exponential_rv{0, (0,), floatX, False}.out, TensorConstant{[ 0.  1.  ... 54. 59.]}, TensorConstant{inf})
Toposort index: 13
Inputs types: [RandomStateType, TensorType(int64, (1,)), TensorType(int64, ()), TensorType(float64, (None,)), TensorType(float64, (39,)), TensorType(float64, ())]
Inputs shapes: ['No shapes', (1,), (), (39,), (39,), ()]
Inputs strides: ['No strides', (8,), (), (8,), (8,), ()]
Inputs values: [RandomState(MT19937) at 0x129E04C40, array([39]), array(11), 'not shown', 'not shown', array(inf)]
Inputs type_num: ['', 7, 7, 12, 12, 12]
Outputs clients: [['output'], []]

Backtrace when the node is created (use Aesara flag 

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 18 seconds.


In [123]:
az.summary(trace, hdi_prob=0.9)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-3.279,0.186,-3.576,-2.967,0.001,0.001,20036.0,23506.0,1.0
beta1,-0.545,0.302,-1.028,-0.036,0.002,0.002,17664.0,23424.0,1.0
censored[0],27.157,28.238,0.000,62.402,0.137,0.105,28314.0,16450.0,1.0
censored[1],28.014,27.827,1.002,63.195,0.137,0.104,29036.0,18124.0,1.0
censored[2],31.142,28.292,4.002,66.946,0.141,0.108,30357.0,20039.0,1.0
censored[3],34.122,28.119,7.003,70.286,0.139,0.102,26036.0,15214.0,1.0
censored[4],37.382,28.214,10.002,73.085,0.144,0.105,24422.0,14932.0,1.0
censored[5],40.982,27.976,14.003,76.537,0.139,0.101,26403.0,16919.0,1.0
censored[6],45.158,28.063,18.001,80.428,0.137,0.102,28217.0,17902.0,1.0
censored[7],50.015,27.883,23.000,85.596,0.132,0.100,32005.0,18188.0,1.0
